In [1]:
#------ Dependencies 

from bs4 import BeautifulSoup as bs
import requests
from splinter import Browser
import time
import pandas as pd

In [2]:
#------ Initialize the browser

def init_browser():   
    executable_path = {"executable_path": "chromedriver.exe"}
    return Browser("chrome", **executable_path, headless=True)

## NASA Mars site

In [3]:
#------ Scrapping the NASA 

def nasamars():
    
    browser = init_browser()

    # Visit NASA Mars site
    url1 = "https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest"
    browser.visit(url1)

    time.sleep(1)

    # Scrape page into Soup
    html = browser.html
    soup = bs(html, "html.parser")

    # All news

    all_news = soup.find('div',class_='grid_layout')

    # title and teaser content of the news

    news_title = all_news.find('div', class_='content_title').text.lstrip()
    news_p = all_news.find('div',class_='article_teaser_body').text.lstrip()

    # Quite the browser after scraping
    browser.quit()
    return news_title, news_p




## Space Images

In [4]:

def spaceimages():
    
    browser = init_browser()

    # Visit Space images -> Mars
    url2 = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
    browser.visit(url2)
    space_page='https://www.jpl.nasa.gov'
    time.sleep(1)

    # Scrape page into Soup
    html = browser.html
    soup = bs(html, "html.parser")

    # All news

    img_div = soup.find('div',class_='img')
    img = img_div.find('img', class_='thumb')["src"]
    url_img = space_page+img
    url_img
    return url_img
    # Quite the browser after scraping
    browser.quit()


## Mars Weather

In [5]:
def marsweather():
    
    browser = init_browser()

    # Visit Mars Weather 
    url3 = "https://twitter.com/marswxreport?lang=en"
    browser.visit(url3)

    time.sleep(1)

    # Scrape page into Soup
    html = browser.html
    soup = bs(html, "html.parser")

    # All news
    mars_twitter = soup.find('div',class_='css-901oao r-hkyrab r-1qd0xha r-a023e6 r-16dba41 r-ad9z0x r-bcqeeo r-bnwqim r-qvutc0')
    mars_weather = mars_twitter.find('span',class_='css-901oao css-16my406 r-1qd0xha r-ad9z0x r-bcqeeo r-qvutc0').text
    return mars_weather

    # Quite the browser after scraping
    browser.quit()


## Mars Facts

In [6]:
def marsfacts():
    url4 = 'https://space-facts.com/mars/'
    tables = pd.read_html(url4)

    mars_facts=tables[0]
    #print(mars_facts)
    return mars_facts


In [7]:
mars_facts = pd.DataFrame(data =marsfacts())


In [8]:
mars_facts.rename(columns={
    0:'Names',
    1:'Values'
},inplace=True)

mars_facts.set_index('Names',inplace=True)
mars_facts_table= mars_facts.to_html()


## Mars Hemispheres

In [9]:
def marshemispheres():
    browser = init_browser()

    # Visit Mars Hmispheres page 
    url5 = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
    browser.visit(url5)

    time.sleep(1)

    # Scrape page into Soup
    html = browser.html
    soup = bs(html, "html.parser")
    full_img='https://astropedia.astrogeology.usgs.gov/download/'
    extension='.tif/full.jpg'

    # All items
    hemispheres_div = soup.find_all('div',class_='item')

    title=[]
    img_url=[]
    for div in hemispheres_div:
        des = div.find('div',class_='description')
        a = des.find('a', class_='itemLink product-item')
        link =a["href"]
        head=a.find('h3').text
        img_link = full_img+link.strip('/search/map')+extension
        title.append(head)
        img_url.append(img_link)

    hemisphere_image_urls =  [{'title':title,'url_img': img_url} for (title,img_url) in zip(title,img_url)]
    return hemisphere_image_urls

    # Quite the browser after scraping
    browser.quit()


In [10]:
def scrape():
    nasa_mars = nasamars()
    news_title = nasa_mars[0]
    news_p = nasa_mars[1]
    
    url_img = spaceimages()    
    
    mars_weather = marsweather()    
    
    mars_facts = marsfacts()   
    
    hemisphere_image_urls = marshemispheres()
    
    scrape ={
        'news_title':news_title,
        'news_teaser':news_p,
        'url_img':url_img,
        'mars_weather':mars_weather,
        'mars_facts': mars_facts_table,
        'hemisphere_image_urls':hemisphere_image_urls
    }
    return scrape
dict_scrape = scrape()

In [11]:
dict_scrape

{'news_title': "NASA Wins 4 Webbys, 4 People's Voice Awards",
 'news_teaser': 'Winners include the JPL-managed "Send Your Name to Mars" campaign, NASA\'s Global Climate Change website and Solar System Interactive.',
 'url_img': 'https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA23910-640x350.jpg',
 'mars_weather': 'InSight sol 525 (2020-05-18) low -93.6ºC (-136.5ºF) high -0.4ºC (31.3ºF)\nwinds from the WNW at 4.8 m/s (10.8 mph) gusting to 18.5 m/s (41.4 mph)\npressure at 7.10 hPa',
 'mars_facts': '<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Values</th>\n    </tr>\n    <tr>\n      <th>Names</th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Equatorial Diameter:</th>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>Polar Diameter:</th>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <th>